In [29]:
[1,2,3] + [4,5,6]

[1, 2, 3, 4, 5, 6]

In [13]:
import numpy as np

lambda_mat = np.full((3,3), 0)
lambda_mat[1,1] = 1
print(lambda_mat)

[[0 0 0]
 [0 1 0]
 [0 0 0]]


In [3]:
from Model.InteractionNode import InteractionNode

In [17]:
print("===")
print("Product={}, PriceLevel={}, Bought={}, #Units={}".format(2,3,"Si",45))


l21 = InteractionNode(3, 4, 0, 0, [])
l22 = InteractionNode(2, 1, 0, 0, [])
l1 = InteractionNode(1, 2, 1, 34, [l21,l22])

l1.printInteractions()

===
Product=2, PriceLevel=3, Bought=Si, #Units=45

USER INTERACTIONS: 
╚═ Product=1, PriceLevel=2, Bought=Yes, #Units=34
  ╚═ Product=3, PriceLevel=4, Bought=No, #Units=0
  ╚═ Product=2, PriceLevel=1, Bought=No, #Units=0


In [19]:
import numpy as np
np.random.uniform(low=0.0, high=1.0, size=None)

0.27985094518957787

In [ ]:
import pprint
import sys
import unittest
from Model.GraphProbabilities import GraphProbabilities
from Model.Product import Product
from Model.UserClass import UserClass
from Model.constants import PROBABILITY_MATRIX, SECONDARY_PRODUCTS

id = 0
conversionRate = [
    [1, 0.3, 0.9, 0.2],
    [1, .2, .56, .12],
    [1, .34, .57, .88],
    [1, .2, .56, .12],
    [1, 0.9, 0.17, 0.96]
]

PROBABILITY_MATRIX_3 = [[0, 1, 1, 0, 0],
                      [0.8, 0, 0, 0.9, 0],
                      [0, 1, 0, 0.7, 0],
                      [0.9, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0]]
clickProbability = GraphProbabilities(PROBABILITY_MATRIX_3)
alphas = [1,0,0,0,0]

n_bought_mean = 2
n_bought_variance = 1
n_user_mean = 10
n_user_variance = 1

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]

Lambda = 1

userClass = UserClass(id=id, conversionRate=conversionRate, clickProbability=clickProbability, alphas=alphas,
                        Lambda=Lambda, n_bought_variance=n_bought_variance, n_bought_mean=n_bought_mean,
                        n_user_mean=n_user_mean, n_user_variance=n_user_variance, productList=productList,
                        features_generator=[{"name":"Over 18","probability":0.6},{"name":"Male","probability":0.9}],
                        debug=False)

# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(userClass.__dict__)

currentPrice = [0, 0, 0, 0, 0]
userClass.setCurrentPrice(currentPrice)

userInteractions = userClass.generateEpisode()
userInteractions.printInteractions()

In [ ]:
def yummy(time, env):
    print("Time: {}".format(time))
    print("Env: {}".format(env.Lambda))
from Environment import Environment
e = Environment(0.3, [userClass])
e.addTimeListener(2, yummy)
e.addTimeListener(2, yummy)
e.round()

In [34]:
class StepNode:
    def __init__(self, product, paths=[], graph_prob=None):
        self.paths=paths
        self.product = product
        self.feasible_paths = []
        self.graph_prob = graph_prob
        productIndex = product
        for i in range(0,len(paths)):
            path = np.array(paths[i])
            if np.any(path[:] == product) == False:
                path = np.append(path, productIndex)
                self.feasible_paths.append(path.tolist())
        
        
    def computeProbability(self):
        paths_prob = []
        # Compute probability of each path
        for i in range(0,len(self.feasible_paths)):
            path = self.feasible_paths[i]
            prob = 1
            print("Path indexes for product {} : {}".format(self.product, path))
            for k in range(0,len(path)-1):
                prob = prob * self.graph_prob.getEdgeProbability(path[k],path[k+1])
            paths_prob.append(prob)
        # Merge all paths prob by using OR, P(path1 OR path2) = 1 - (1 - P(path1)) * (1 - P(path2))
        inverse_prob = 1
        for i in range(0,len(paths_prob)):
            inverse_prob = inverse_prob * (1 - paths_prob[i])
        return 1 - inverse_prob

    def merge(self, node):
        if self.product != node.product:
            return
        # Concatenate the two lists
        self.feasible_paths += node.feasible_paths
        return self

    def expand(self):
        n_products = self.graph_prob.shape()[0]
        next_nodes = []
        for i in range(0,n_products):
            if self.graph_prob.getEdgeProbability(self.product,i) > 0 :
                next_nodes.append(StepNode(i,paths=self.feasible_paths,graph_prob=self.graph_prob))
        return next_nodes
    
    def isFeasible(self):
        return len(self.feasible_paths) != 0
        

In [37]:
from Model.GraphProbabilities import GraphProbabilities
from Model.Product import Product
import numpy as np


class EvaluationAlgorithm:
    def __init__(self, products_list=[], click_prob_matrix=None, lambda_prob=0.5, conversion_rates=[], 
                alphas=[], margins=[], verbose=False):
        assert len(products_list) == len(conversion_rates) and len(products_list) == len(alphas)
        assert len(products_list) == len(margins)
        assert click_prob_matrix is not None

        self.products_list = products_list
        self.lambda_prob = lambda_prob
        self.conversion_rates = np.array(conversion_rates)
        self.n_products = len(products_list)
        self.alphas = np.array(alphas)
        self.margins = np.array(margins)
        self.verbose = verbose
        lambda_mat = np.full((self.n_products,self.n_products), 0, dtype=float)
        for i in range(0,len(products_list)):
            assert i == products_list[i].getProductNumber()
            endFirst = products_list[i].getSecondaryProduct(0)
            endSecond = products_list[i].getSecondaryProduct(1)
            # if verbose: print("Coordinates {},{} ----- {},{}".format(i,endFirst,i,endSecond))
            lambda_mat[i][endFirst] = 1
            lambda_mat[i][endSecond] = lambda_prob
        #if verbose: print(lambda_mat)
        
        conv_mat = []
        for i in range(0,len(conversion_rates)):
            conv_mat.append(np.full((len(products_list)), conversion_rates[i]).tolist())
        #if verbose: print(conv_mat)
        
        w_matrix = GraphProbabilities(click_prob_matrix)
        lambda_matrix = GraphProbabilities(lambda_mat.tolist())
        conversion_matrix = GraphProbabilities(conv_mat)

        y_matrix = w_matrix.multiplyElementWise(lambda_matrix)
        self.y_matrix = y_matrix.multiplyElementWise(conversion_matrix)
        if verbose: print(self.y_matrix.weightMatrix)

    def computeSingleProduct(self, product):
        firstNode = StepNode(product, [np.array([], dtype=int)], graph_prob=y_matrix)
        nodes=[firstNode]
        inverse_prob = np.full((len(self.products_list)), 1)
        # Iterate for #steps times
        for i in range(0, len(self.products_list)-1):
            # Next nodes
            product_nodes = np.full((len(self.products_list)), None)
            for k in range(0,len(nodes)):
                node = nodes[k]
                following = node.expand()
                for j in range(0,len(following)):
                    if following[j].isFeasible() == True:
                        if product_nodes[following[j].product] is not None:
                            product_nodes[following[j].product].merge(following[j])
                        else:
                            product_nodes[following[j].product] = following[j]
            
            # Remove None elements
            existing_nodes = product_nodes[product_nodes != np.array(None)]
            reached_nodes = ""
            for k in range(0,len(existing_nodes)):
                index = existing_nodes[k].product
                reached_nodes = reached_nodes + str(index) + "; "
                # existing_nodes[k].computeProbability() is the probability of visiting "index" in (i+1)-steps
                inverse_prob[index] = inverse_prob[index] * (1 - existing_nodes[k].computeProbability())
            if self.verbose: print("Reached nodes in {}-step: {}".format(i+1,reached_nodes))
            nodes = existing_nodes
        # Probability of visiting product
        return  1 - inverse_prob 

    def computeMargin(self):
        single_margins = np.full((len(self.products_list)), 0)
        for i in range(0,len(self.products_list)):
            visiting_prob = self.computeSingleProduct(i)
            # Margin if alpha = [1 0 0 0 0]
            single_margins[i] = np.multiply(visiting_prob,np.multiply(self.margins,self.conversion_rates)).sum()
            if self.verbose: print("Expected value margin for product {} as starting is {} \n".format(i, single_margins[i]))
        # Weight the single margin by alpha
        return np.multiply(single_margins, alphas).sum()
            

In [40]:
from Model.GraphProbabilities import GraphProbabilities
from Model.Product import Product
from Model.UserClass import UserClass
import numpy as np

CLICK_PROB = [[0, .8, .7, 0, 0],
            [0, 0, 0, .9, .8],
            [0, 1, 0, .75, 0],
            [.9, 0, 0, 0, .82],
            [0, 0, .87, .92, 0]]
L = 0.5
LAMBDA_MATRIX = [[0, 1, L, 0, 0],
                [0, 0, 0, L, 1],
                [0, L, 0, 1, 0],
                [L, 0, 0, 0, 1],
                [0, 0, 1, L, 0]]
conversionRate = [
    [0.9, 0.9, 0.9, 0.9, 0.9],
    [.62, .62, .62, .62, .62],
    [.82, .82, .82, .82, .82],
    [.56, .56, .56, .56, .56],
    [.92, 0.92, 0.92, 0.92, .92]
]
convRates = [.9, .62, .82, .56, .92]
alphas = [0.2, 0.1, 0.3, 0.1, 0.3]
SECONDARY_PRODUCTS = {'0': [1, 2], '1': [4, 3], '2': [3, 1], '3': [4, 0], '4': [2, 3]}
margins = [12,7,9,19,5]

w_matrix = GraphProbabilities(CLICK_PROB)
lambda_matrix = GraphProbabilities(LAMBDA_MATRIX)
conv_matrix = GraphProbabilities(conversionRate)
y_matrix = w_matrix.multiplyElementWise(lambda_matrix)
y_matrix = y_matrix.multiplyElementWise(conv_matrix)

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]
eval = EvaluationAlgorithm(products_list=productList, click_prob_matrix=CLICK_PROB, lambda_prob=0.7, conversion_rates=convRates,
                    alphas=alphas, margins=margins, verbose=True)
print("Expected profit over all interactions: {}".format(eval.computeMargin()))

profitSingle = np.multiply(np.array(alphas),np.multiply(np.array(margins),np.array(convRates))).sum()
print("Expected profit over single products: {}".format(profitSingle))

[[0.0, 0.6480000000000001, 0.19845, 0.0, 0.0], [0.0, 0.0, 0.0, 0.121086, 0.30752], [0.0, 0.23533999999999997, 0.0, 0.5043, 0.0], [0.09878400000000002, 0.0, 0.0, 0.0, 0.25715200000000005], [0.0, 0.0, 0.736368, 0.2725408, 0.0]]
Path indexes for product 1 : [0, 1]
Path indexes for product 2 : [0, 2]
Reached nodes in 1-step: 1; 2; 
Path indexes for product 1 : [0, 2, 1]
Path indexes for product 3 : [0, 1, 3]
Path indexes for product 3 : [0, 2, 3]
Path indexes for product 4 : [0, 1, 4]
Reached nodes in 2-step: 1; 3; 4; 
Path indexes for product 2 : [0, 1, 4, 2]
Path indexes for product 3 : [0, 2, 1, 3]
Path indexes for product 3 : [0, 1, 4, 3]
Path indexes for product 4 : [0, 2, 1, 4]
Path indexes for product 4 : [0, 1, 3, 4]
Path indexes for product 4 : [0, 2, 3, 4]
Reached nodes in 3-step: 2; 3; 4; 
Path indexes for product 2 : [0, 1, 3, 4, 2]
Path indexes for product 3 : [0, 1, 4, 2, 3]
Path indexes for product 3 : [0, 2, 1, 4, 3]
Path indexes for product 4 : [0, 2, 1, 3, 4]
Reached node